In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -U torch transformers tokenizers accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Reference
"""
from transformers import AutoModelForSeq2SeqLM
+ from peft import PeftModel, PeftConfig

peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
+ model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = model.to(device)
model.eval()
"""

In [ ]:
!unzip -qq '/content/EleutherAI.zip'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import transformers
from peft import PeftModel, PeftConfig


model_id = "EleutherAI/polyglot-ko-12.8b"
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

peft_model_id = f"{model_id}_{config.peft_type}_{config.task_type}"
print("-----peft_model_id-----")
print(peft_model_id)

'''
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
'''

config = PeftConfig.from_pretrained(peft_model_id)
print('-----peftconfig config-----')
print(config)


print('-----LOAD MODEL-----')
# loaded_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
# loaded_model = AutoModelForCausalLM.from_pretrained(peft_model_id, quantization_config=bnb_config, device_map={"":0})
loaded_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                                    quantization_config=bnb_config)

model = PeftModel.from_pretrained(loaded_model, peft_model_id)

device = torch.device('cuda:0')
model = model.to(device)
model.eval()


print('-----LOAD TOKENIZER-----')
# loaded_tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

-----peft_model_id-----
EleutherAI/polyglot-ko-12.8b_LORA_CAUSAL_LM
-----peftconfig config-----
LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='EleutherAI/polyglot-ko-12.8b', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=8, target_modules=['query_key_value'], lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)
-----LOAD MODEL-----


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

-----LOAD TOKENIZER-----


Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [ ]:
'''
device = torch.device('cuda:0')
model = model.to(device)
model.eval()

'''

'''
# testing
inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])
'''

In [ ]:
# Answer Generation Function

def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        top_k=50,
        top_p =0.95,
        num_beams=2,
    )
    # print(tokenizer.decode(gened[0]))
    # gened[0] : embedding tensor 형식  >> decode로 decoding해서 다시 string으로
    gen_text = tokenizer.decode(gened[0], skip_special_tokens=True)
    # print(gen_text)

    return gen_text

In [ ]:
prompt = "'성별 : 남자, 나이 : 25, 계절 : 가을, 상황 : 여행, 상의 색 : ,하의 색 :  '에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘"

In [ ]:
# prompt = " ' 성별 : 여자 , 나이 : 25 ,날씨 : 온도 10, 습도 30, 강수확률 100% ,상황 : 데이트 ,상의 색 : 파란 ,하의 색 : 흰색 '에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘"
generated_text = gen(prompt)
print('answer :', generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: '성별 : 여자, 나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 '에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘

### 답변:''성별 : 여자, 나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 '에 대한 한 가지의 옷 추천을 키워드로
